#### Stats, Head and Body
The base stats of a fused Pokémon are determined by the base stats of its head and body Pokémon. The formula is a weighted average with a bias towards the **body Pokémon for physical stats** (Atk, Def, Spd) and a bias towards the **head Pokémon for special stats (HP, Sp.Atk, Sp.Def)**.

- ATK, DEF, SPD

fusionStat = 2 * bodyStat / 3 + headStat / 3

- HP, SP.DEF, SP.ATK

fusionStat = 2 * headStat / 3 + bodyStat/3


#### Name, Nature, and Moves
You will be able to choose a name, nature, and set of moves based on what either Pokémon has at the time. The Move Reminder found on the top floor of Pokémon centers can teach moves from either original Pokémon.

Nature can be changed in the far southwest of Cinnabar Island.

#### Ability
A fusion has access to any of the abilities of the Fused pokemon. When fusing, You are only able to choose between the two abilities that the Parent Pokemon had at the time of fusion. Ex: fusing Politoed with damp, and Glaceon with snow cloak will only show those two abilities, regardless of fusion order.

#### EVs and IVs
Fusion keeps the EVs of the body and discards the EVs of the head.

If using DNA splicers, an average of both Pokémon's IVs is used. If using super splicers, the higher IV in each stat is used.

#### Typing

The fused Pokémon will have its head's primary type and its body's secondary type. If the body's Pokémon doesn't have a secondary type, its primary type will be used instead.

If the head is already providing the element the body wants to provide, The body will provide its primary type instead. A Grimer/Oddish for example is Poison/Grass. Oddish normally provides Poison as a body, but Grimer already provides Poison; so to avoid redundancy, Oddish instead provides its primary type, Grass.

In [1]:
import numpy as np
import pandas as pd

pokedex = pd.read_csv("data/CompleteInfiniteFusionDex.csv")

type_chart = pd.read_csv("data/typingchart.csv")

type_chart = type_chart.set_index("Attacking")
type_chart.columns = type_chart.columns.str.lower()
type_chart.index = type_chart.index.str.lower()

In [2]:
def pkm_query(pokemon:str):
    return pokedex.loc[pokedex.Species == pokemon.lower()]

In [3]:
def weighted_avg(stat1,stat2):
    return int((2*stat1+stat2)/3)

In [4]:
def calc_fusion_stats(head, body):
    fusion_head = pkm_query(head)
    fusion_body = pkm_query(body)
    fusion_stats = {}
    # Pandas doc recommends using to_numpy() instead of values
    fusion_stats["hp"] = weighted_avg(fusion_head["hp"].values[0],fusion_body["hp"].values[0])
    fusion_stats["atk"] = weighted_avg(fusion_body["attack"].values[0],fusion_head["attack"].values[0])
    fusion_stats["def"] = weighted_avg(fusion_body["defense"].values[0],fusion_head["defense"].values[0])
    fusion_stats["spatk"] = weighted_avg(fusion_head["sp_attack"].values[0],fusion_body["sp_attack"].values[0])
    fusion_stats["spdef"] = weighted_avg(fusion_head["sp_defense"].values[0],fusion_body["sp_defense"].values[0])
    fusion_stats["spe"] = weighted_avg(fusion_body["speed"].values[0],fusion_head["speed"].values[0])
    fusion_stats["total"] = sum(fusion_stats.values())
    return fusion_stats

In [5]:
def calc_fusion_typing(head,body):
    fusion_head = pkm_query(head)
    fusion_body = pkm_query(body)
    fusion_types = {}
    fusion_types["Type1"] = fusion_head["Type1"].values[0]
    if fusion_body["Type2"].values[0] == fusion_head["Type1"].values[0]:
        fusion_types["Type2"] = fusion_body["Type1"].values[0]
    else:
        fusion_types["Type2"] = fusion_body["Type2"].values[0]
    return fusion_types

In [6]:
def calc_fusion_weakness(fusion_typing:dict):
    if fusion_typing['Type1'] == fusion_typing['Type2']:
        pkm_weak = type_chart[fusion_typing['Type1']]
    else:
        pkm_weak = type_chart[fusion_typing['Type1']]*type_chart[fusion_typing['Type2']]

    fusion_weak = {}
    for value in pkm_weak:
        if value not in fusion_weak:
            fusion_weak[str(value)] = pkm_weak[pkm_weak == value].index.tolist()

    return fusion_weak

In [27]:
from dataclasses import dataclass
@dataclass
class Fusion:
    head:str
    body:str
    types:dict
    stats:dict
    
    def __init__(self,head,body):
        self.head = head
        self.body = body
        
        self.types = calc_fusion_typing(head,body)
        self.stats = calc_fusion_stats(head,body)
        
    def print_weakness(self):
        weak = calc_fusion_weakness(self.types)
        keys = list(weak.keys())
        keys.sort(reverse=True)
        for key in keys:
            print(f"{key} ({len(weak[key])}):{weak[key]}")
    
    def possible_abilities(self):
        head = pkm_query(self.head)
        body = pkm_query(self.body)

        abilites = {}
        abilites['regular'] = [head.ability_1.values[0], head.ability_2.values[0], body.ability_1.values[0], body.ability_2.values[0]]
        abilites['hidden'] = [head.ability_hidden.values[0], body.ability_hidden.values[0]]
        for key in abilites.keys():
            abilites[key] = [x for x in abilites[key] if x != 'none']
        return abilites

In [28]:
def calc_fusion(pokemon1:str, pokemon2:str):
    return Fusion(pokemon1,pokemon2)

## Fusion Calculator

In [29]:
def fusion_calculator_simple(pokemon1:str,pokemon2:str):
    fusions = [calc_fusion(pokemon1, pokemon2),calc_fusion(pokemon2, pokemon1)]
    print("Typing and Stats")
    for fusion in fusions:
        print(f"Head: {fusion.head} | Body: {fusion.body}")
        print(f"{fusion.types}")
        print(f"{fusion.stats}")
        print("")
    print("Weakness")
    if set(fusions[0].types.values()) == set(fusions[1].types.values()):
        print("Fusions share the same typing")
        fusions[0].print_weakness()
    else:
        for fusion in fusions:
            print(f"Head: {fusion.head} | Body: {fusion.body}")
            fusion.print_weakness()
        

In [36]:
def fusion_calculator(pokemon1:str,pokemon2:str):
    fusions = [calc_fusion(pokemon1, pokemon2),calc_fusion(pokemon2, pokemon1)]
    stats_diff = {}
    for key in fusions[0].stats.keys():
        stats_diff[key] = fusions[0].stats[key] - fusions[1].stats[key]
        stats_diff[key] = str(stats_diff[key]) if stats_diff[key] < 0 else f"+{str(stats_diff[key])}"

    stats_print = {}
    for key in fusions[0].stats.keys():
            stats_print[key] = [fusions[0].stats[key],stats_diff[key],fusions[1].stats[key]]
    
    print("Typing and Stats")
    print(f"{'-'*20}")
    print(f"Head: {fusions[0].head} | Body: {fusions[0].body} | Typing: {list(fusions[0].types.values())}")
    sio = ['hp','atk','def','spatk','spdef','spe','total'] #stats in order
    aw = 7 #alignment width
    print(f"{sio[0]:^7}{sio[1]:^7}{sio[2]:^7}{sio[3]:^7}{sio[4]:^7}{sio[5]:^7}{sio[6]:^7}") #^ central alignment, < left, > right
    for i in range(3):
        print(f"{stats_print[sio[0]][i]:^7}{stats_print[sio[1]][i]:^7}{stats_print[sio[2]][i]:^7}{stats_print[sio[3]][i]:^7}{stats_print[sio[4]][i]:^7}{stats_print[sio[5]][i]:^7}{stats_print[sio[6]][i]:^7}")
    print(f"Head: {fusions[1].head} | Body: {fusions[1].body} | Typing: {list(fusions[1].types.values())}")
    print("")
    print(f"{'='*80}\nWeakness\n{'-'*20}")
    if set(fusions[1].types.values()) == set(fusions[0].types.values()):
        print("Fusions share the same typing")
        fusions[1].print_weakness()
    else:
        for fusion in fusions:
            print(f"Head: {fusion.head} | Body: {fusion.body} | Typing: {list(fusion.types.values())}")
            fusion.print_weakness()
            print(f"{'-'*20}")
    print("")
    print(f"{'='*80}\nAbilities\n{'-'*20}")
    fusion_abilities = fusions[0].possible_abilities()
    for key in fusion_abilities.keys():
        print(f"{key}: {fusion_abilities[key]}")
    

In [38]:
fusion_calculator("infernape","leafeon")

Typing and Stats
--------------------
Head: infernape | Body: leafeon | Typing: ['fire', 'grass']
  hp     atk    def   spatk  spdef   spe   total 
  72     108    110    89     69     99     547  
  +4     +2     +20    +15    +2     -4     +39  
  68     106    90     74     67     103    508  
Head: leafeon | Body: infernape | Typing: ['grass', 'fighting']

Weakness
--------------------
Head: infernape | Body: leafeon | Typing: ['fire', 'grass']
2.0 (3):['poison', 'flying', 'rock']
1.0 (11):['normal', 'fire', 'water', 'ice', 'fighting', 'ground', 'psychic', 'bug', 'ghost', 'dragon', 'dark']
0.5 (3):['electric', 'steel', 'fairy']
0.25 (1):['grass']
--------------------
Head: leafeon | Body: infernape | Typing: ['grass', 'fighting']
4.0 (1):['flying']
2.0 (5):['fire', 'ice', 'poison', 'psychic', 'fairy']
1.0 (6):['normal', 'fighting', 'bug', 'ghost', 'dragon', 'steel']
0.5 (6):['water', 'electric', 'grass', 'ground', 'rock', 'dark']
--------------------

Abilities
--------------------

In [40]:
fusion_calculator("venusaur","typhlosion")

Typing and Stats
--------------------
Head: venusaur | Body: typhlosion | Typing: ['grass', 'fire']
  hp     atk    def   spatk  spdef   spe   total 
  79     83     79     103    95     93     532  
  +1     +1     -2     -3     +5     +7     +9   
  78     82     81     106    90     86     523  
Head: typhlosion | Body: venusaur | Typing: ['fire', 'poison']

Weakness
--------------------
Head: venusaur | Body: typhlosion | Typing: ['grass', 'fire']
2.0 (3):['poison', 'flying', 'rock']
1.0 (11):['normal', 'fire', 'water', 'ice', 'fighting', 'ground', 'psychic', 'bug', 'ghost', 'dragon', 'dark']
0.5 (3):['electric', 'steel', 'fairy']
0.25 (1):['grass']
--------------------
Head: typhlosion | Body: venusaur | Typing: ['fire', 'poison']
4.0 (1):['ground']
2.0 (3):['water', 'psychic', 'rock']
1.0 (6):['normal', 'electric', 'flying', 'ghost', 'dragon', 'dark']
0.5 (5):['fire', 'ice', 'fighting', 'poison', 'steel']
0.25 (3):['grass', 'bug', 'fairy']
--------------------

Abilities
--------